# Visualizing a Gensim model

To illustrate how to use [`pyLDAvis`](https://github.com/bmabey/pyLDAvis)'s gensim [helper funtions](https://pyldavis.readthedocs.org/en/latest/modules/API.html#module-pyLDAvis.gensim) we will create a model from the [20 Newsgroup corpus](http://qwone.com/~jason/20Newsgroups/). Minimal preprocessing is done and so the model is not the best, the goal of this notebook is to demonstrate the the helper functions.

## Downloading the data

In [1]:
%%bash
mkdir -p data
pushd data
if [ -d "20news-bydate-train" ]
then
  echo "The data has already been downloaded..."
else
  wget http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
  tar xfv 20news-bydate.tar.gz
  rm 20news-bydate.tar.gz
fi
echo "Lets take a look at the groups..."
ls 20news-bydate-train/
popd

~/Desktop/Honours-LDA/notebooks/data ~/Desktop/Honours-LDA/notebooks
20news-bydate-test/
20news-bydate-test/alt.atheism/
20news-bydate-test/alt.atheism/53265
20news-bydate-test/alt.atheism/53339
20news-bydate-test/alt.atheism/53260
20news-bydate-test/alt.atheism/53340
20news-bydate-test/alt.atheism/53333
20news-bydate-test/alt.atheism/53302
20news-bydate-test/alt.atheism/53313
20news-bydate-test/alt.atheism/53293
20news-bydate-test/alt.atheism/53297
20news-bydate-test/alt.atheism/53315
20news-bydate-test/alt.atheism/53320
20news-bydate-test/alt.atheism/53324
20news-bydate-test/alt.atheism/53328
20news-bydate-test/alt.atheism/53325
20news-bydate-test/alt.atheism/53322
20news-bydate-test/alt.atheism/53326
20news-bydate-test/alt.atheism/53261
20news-bydate-test/alt.atheism/53327
20news-bydate-test/alt.atheism/53329
20news-bydate-test/alt.atheism/53321
20news-bydate-test/alt.atheism/53068
20news-bydate-test/alt.atheism/53338
20news-bydate-test/alt.atheism/53257
20news-bydate-test/alt.athei

--2017-12-30 15:42:46--  http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 108.20.215.97
Connecting to qwone.com (qwone.com)|108.20.215.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘20news-bydate.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  192K 73s
    50K .......... .......... .......... .......... ..........  0%  400K 54s
   100K .......... .......... .......... .......... ..........  1% 1.18M 40s
   150K .......... .......... .......... .......... ..........  1% 1.17M 33s
   200K .......... .......... .......... .......... ..........  1%  647K 30s
   250K .......... .......... .......... .......... ..........  2% 1.16M 27s
   300K .......... .......... .......... .......... ..........  2% 1.14M 25s
   350K .......... .......... .......... .......... ..........  2% 1.19M 23s
   400K .......... .......... .......... .......... ....

## Exploring the dataset

Each group dir has a set of files:

In [2]:
ls -lah data/20news-bydate-train/sci.space | tail  -n 5

-rw-r--r--  1 ramona ramona 1.5K Mar 18  2003 61250
-rw-r--r--  1 ramona ramona  889 Mar 18  2003 61252
-rw-r--r--  1 ramona ramona 1.2K Mar 18  2003 61264
-rw-r--r--  1 ramona ramona 1.7K Mar 18  2003 61308
-rw-r--r--  1 ramona ramona 1.4K Mar 18  2003 61422


Lets take a peak at one email:

In [3]:
!head data/20news-bydate-train/sci.space/61422 -n 20

From: ralph.buttigieg@f635.n713.z3.fido.zeta.org.au (Ralph Buttigieg)
Subject: Why not give $1 billion to first year-lo
Organization: Fidonet. Gate admin is fido@socs.uts.edu.au
Lines: 34

Original to: keithley@apple.com
G'day keithley@apple.com

21 Apr 93 22:25, keithley@apple.com wrote to All:

 kc> keithley@apple.com (Craig Keithley), via Kralizec 3:713/602


 kc> But back to the contest goals, there was a recent article in AW&ST
about a
 kc> low cost (it's all relative...) manned return to the moon.  A General
 kc> Dynamics scheme involving a Titan IV & Shuttle to lift a Centaur upper
 kc> stage, LEV, and crew capsule.  The mission consists of delivering two
 kc> unmanned payloads to the lunar surface, followed by a manned mission.
 kc> Total cost:  US was $10-$13 billion.  Joint ESA(?)/NASA project was


## Loading the tokenizing the corpus

In [4]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [5]:
# quick and dirty....
EMAIL_REGEX = re.compile(r"[a-z0-9\.\+_-]+@[a-z0-9\._-]+\.[a-z]*")
FILTER_REGEX = re.compile(r"[^a-z '#]")
TOKEN_MAPPINGS = [(EMAIL_REGEX, "#email"), (FILTER_REGEX, ' ')]

def tokenize_line(line):
    res = line.lower()
    for regexp, replacement in TOKEN_MAPPINGS:
        res = regexp.sub(replacement, res)
    return res.split()
    
def tokenize(lines, token_size_filter=2):
    tokens = fp.mapcat(tokenize_line, lines)
    return [t for t in tokens if len(t) > token_size_filter]
    

def load_doc(filename):
    group, doc_id = filename.split('/')[-2:]
    with open(filename, errors='ignore') as f:
        doc = f.readlines()
    return {'group': group,
            'doc': doc,
            'tokens': tokenize(doc),
            'id': doc_id}


docs = pd.DataFrame(list(map(load_doc, glob('data/20news-bydate-train/*/*')))).set_index(['group','id'])
docs.head()

doc  \
group       id                                                         
alt.atheism 53304  [From: I3150101@dbstu1.rz.tu-bs.de (Benedikt R...   
            53308  [From: I3150101@dbstu1.rz.tu-bs.de (Benedikt R...   
            51249  [From: mathew <mathew@mantis.co.uk>\n, Subject...   
            51290  [From: bil@okcforum.osrhe.edu (Bill Conner)\n,...   
            51141  [Subject: Re: Speculations\n, From: dgraham@bm...   

                                                              tokens  
group       id                                                        
alt.atheism 53304  [from, #email, benedikt, rosenau, subject, isl...  
            53308  [from, #email, benedikt, rosenau, subject, som...  
            51249  [from, mathew, #email, subject, the, inimitabl...  
            51290  [from, #email, bill, conner, subject, soc, mot...  
            51141  [subject, speculations, from, #email, douglas,...

## Creating the dictionary, and bag of words corpus

In [9]:

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus



In [15]:
dictionary

In [10]:
dictionary, corpus = prep_corpus(docs['tokens'])

Building dictionary...
Building corpus...


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ramona/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
MmCorpus.serialize('newsgroups.mm', corpus)
dictionary.save('newsgroups.dict')

## Fitting the LDA model

In [12]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, passes=1)
                                      
lda.save('newsgroups_50_lda.model')

CPU times: user 2min 12s, sys: 888 ms, total: 2min 12s
Wall time: 1min 22s


## Visualizing the model with pyLDAvis

Okay, the moment we have all been waiting for is finally here!  You'll notice in the visualizaiton that we have a few junk topics that would probably disappear after better preprocessing of the corpus. This is left as an exercises to the reader. :)

In [13]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [14]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/ramona/anaconda3/envs/mlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


## Fitting the HDP model

We could visualize the LDA model with pyLDAvis, in the same maner, we can also visualize gensim HDP models with pyLDAvis.

The difference between HDP and LDA is that HDP is a non-parametric method. Which means you don't need to specify the number of topics, HDP will fit as many topics as it can and find the optimal number of topics by itself.

In [ ]:
%%time
# The optional parameter T here indicates that HDP should find no more than 50 topics
# if there exists any.
hdp = models.hdpmodel.HdpModel(corpus, dictionary, T=50)
                                      
hdp.save('newsgroups_hdp.model')

## Visualizing the HDP model with pyLDAvis

As for the LDA model, you only need to give your model, the corpus and the dictionary associated to prepare the visualization.

In [ ]:
vis_data = gensimvis.prepare(hdp, corpus, dictionary)
pyLDAvis.display(vis_data)